In [1]:
import torch
import numpy as np

In [11]:
# Khởi tạo Tensor
# Từ mảng 
data = [[1,2], [3,4]]
# Từ numpy array
data = np.array(([1,2], [3,4]), dtype=float)
x_data = torch.tensor(data)
print(x_data)
print()
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
print(rand_tensor)
print(ones_tensor)
print(zeros_tensor)

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

tensor([[0.0202, 0.7060, 0.4320],
        [0.0260, 0.9828, 0.8303]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [21]:
# Thông số của Tensor
tensor = torch.rand((3,4))

print(tensor.shape)
print(tensor.dtype)
print(tensor.device)


torch.Size([3, 4])
torch.float32
cpu


In [22]:
# Phép toán với Tensor

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor = tensor.to(device)
print(tensor.device)

cpu
